In [1]:
import awkward as ak
import numpy as np

from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.analysis_tools import PackedSelection

from coffea.ml_tools.torch_wrapper import torch_wrapper

import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import pickle

### Converting model to TorchScript

In [6]:
 # An instance of your model.
model = torch.load("jetMatcherNN_100_strat_newcuts.pt")
model.eval()

# An example input you would normally provide to your model's forward() method.
example = torch.rand(1, 13)
traced_script_module = torch.jit.trace(model, example)
traced_script_module.save("jetMatcherNN_100_strat_newcuts_traced.pt")

### Events

In [2]:
#TODO: fix path issue
fname = "data/NANOAOD-1500_900.root"
events = NanoEventsFactory.from_root(
    {fname: "Events"},
    schemaclass=NanoAODSchema,
    metadata={"dataset": "DYJets"},
    delayed=False
).events()

/usr/local/lib/python3.10/site-packages/coffea/nanoevents/methods/candidate.py:11: FutureWarning: In version 2024.7.0 (target date: 2024-06-30 11:59:59-05:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


In [3]:
def makeCutSet(x, s, *args):
    return [x[s > a] for a in args]

b_tag_wps = [0.0490, 0.2783, 0.7100]

def createObjects(events):
    good_jets = events.Jet[(events.Jet.pt > 30) & (abs(events.Jet.eta) < 2.4)]
    fat_jets = events.FatJet[(events.FatJet.pt > 30) & (abs(events.FatJet.eta) < 2.4)]
    loose_top, med_top, tight_top = makeCutSet(
        fat_jets, fat_jets.particleNet_TvsQCD, 0.58, 0.80, 0.97
    )
    #loose_W, med_W, tight_W = makeCutSet(
    #    fat_jets, fat_jets.particleNet_WvsQCD, 0.7, 0.94, 0.98
    #)

    #deep_top_wp1, deep_top_wp2, deep_top_wp3, deep_top_wp4 = makeCutSet(
    #    fat_jets, fat_jets.deepTag_TvsQCD, 0.436, 0.802, 0.922, 0.989
    #)
    #deep_W_wp1, deep_W_wp2, deep_W_wp3, deep_W_wp4 = makeCutSet(
    #    fat_jets, fat_jets.deepTag_WvsQCD, 0.458, 0.762, 0.918, 0.961
    #)
    loose_b, med_b, tight_b = makeCutSet(
        good_jets, good_jets.btagDeepFlavB, *(b_tag_wps[x] for x in range(3))
    )

    el = events.Electron
    good_electrons = el[
        (el.cutBased == 4)
        & (el.miniPFRelIso_all < 0.1)
        & (el.pt > 30)
        & (abs(el.eta) < 2.4)
    ]
    mu = events.Muon
    good_muons = mu[
        (mu.mediumId) & (mu.miniPFRelIso_all < 0.2) & (mu.pt > 30) & (abs(mu.eta) < 2.4)
    ]
    events["good_jets"] = good_jets
    events["good_electrons"] = good_electrons
    events["good_muons"] = good_muons

    events["loose_bs"] = loose_b
    events["med_bs"] = med_b
    events["tight_bs"] = tight_b

    events["tight_tops"] = tight_top
    # events["med_tops"] = med_top
    # events["loose_tops"] = loose_top

    # events["tight_Ws"] = tight_W
    # events["med_Ws"] = med_W
    # events["loose_Ws"] = loose_W

    # events["deep_top_wp1"] = deep_top_wp1
    # events["deep_top_wp2"] = deep_top_wp2
    # events["deep_top_wp3"] = deep_top_wp3
    # events["deep_top_wp4"] = deep_top_wp4

    # events["deep_W_wp1"] = deep_W_wp1
    # events["deep_W_wp2"] = deep_W_wp2
    # events["deep_W_wp3"] = deep_W_wp3
    # events["deep_W_wp4"] = deep_W_wp4

    return events

def createSelection(events):
    """Baseline selection for the analysis.
Applies the following selection:
- Jets[0].pt > 300
- 4 <= nJets <= 6
- 0 leptons
- 2 medium bjets, at least one of which is tight
- delta_R(med_bjets[0],med_bjets[1]) > 1
    """

    selection = PackedSelection()
    good_jets = events.good_jets
    fat_jets = events.FatJet
    good_muons = events.good_muons
    good_electrons = events.good_electrons
    loose_b = events.loose_bs
    med_b = events.med_bs
    tight_b = events.tight_bs
    tight_top = events.tight_tops
    # selection = PackedSelection()
    filled_jets = ak.pad_none(good_jets, 4, axis=1)
    top_two_dr = ak.fill_none(filled_jets[:, 0].delta_r(filled_jets[:, 1]), False)

    filled_med = ak.pad_none(med_b, 2, axis=1)
    med_dr = ak.fill_none(filled_med[:, 0].delta_r(filled_med[:, 1]), False)
    # selection.add("trigger", (ak.num(good_jets) >= 4) & (ak.num(good_jets) <= 6))
    if "HLT" in events.fields:
        selection.add("hlt", (events.HLT.PFHT1050 | events.HLT.AK8PFJet360_TrimMass30))
    selection.add("highptjet", (ak.fill_none(filled_jets[:, 0].pt > 300, False)))
    selection.add("jets", ((ak.num(good_jets) >= 4) & (ak.num(good_jets) <= 6)))
    selection.add("0Lep", ((ak.num(good_electrons) == 0) & (ak.num(good_muons) == 0)))
    selection.add("2bjet", (ak.num(med_b) >= 2))
    selection.add("1tightbjet", (ak.num(tight_b) >= 1))
    # selection.add("jet_dr", ((top_two_dr < 4) & (top_two_dr > 2)).to_numpy())
    selection.add("b_dr", (med_dr > 1))
    return selection

In [4]:
events = createObjects(events)
mask = createSelection(events).all("highptjet", "jets", "0Lep", "2bjet", "1tightbjet", "b_dr")
events = events[mask]

### NN class

In [5]:
# jetAssignmentNN.model

In [6]:
scl = open('scaler.pkl', 'rb')
scaler = pickle.load(scl)

/usr/local/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 0.24.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
jets = events.good_jets
flat_jets = ak.flatten(jets)

m3 = jets[:,1:4].sum()
m4 = jets[:,0:4].sum()

ones = ak.ones_like(jets.pt)

imap = {
    "features": {
        "jetOrdinality":    ak.flatten(ak.local_index(jets, axis=1)),
        "jetPT": 		    flat_jets.pt,
        "jetEta": 		    flat_jets.eta,
        "jetPhi": 		    flat_jets.phi,
        "jetBScore":    	flat_jets.btagDeepFlavB,
        "m3M": 			    ak.flatten(ones * m3.mass),
        "m3PT": 		    ak.flatten(ones * m3.pt),
        "m3Eta": 		    ak.flatten(ones * m3.eta),
        "m3Phi": 	        ak.flatten(ones * m3.phi),
        "m4M": 			    ak.flatten(ones * m4.mass),
        "m4PT":		        ak.flatten(ones * m4.pt),
        "m4Eta":		    ak.flatten(ones * m4.eta),
        "m4Phi":	        ak.flatten(ones * m4.phi)
    }
}

imap_concat = ak.concatenate([x[:, np.newaxis] for x in imap['features'].values()], axis=1)
imap_scaled = (imap_concat - scaler.mean_) / scaler.scale_
retmap = {
    "features": [imap_scaled]
}

In [8]:
retmap["features"]

[<Array [[-1.36, 1.22, ..., 0.168, 1.09], ...] type='20510 * 13 * float64'>]

In [9]:
imap_scaled

<Array [[-1.36, 1.22, ..., 0.168, 1.09], ...] type='20510 * 13 * float64'>

In [4]:
class jetAssignmentNN(torch_wrapper):
    def prepare_awkward(self,events):
        # ak = self.get_awkward_lib(events)
        jets = events.good_jets
        flat_jets = ak.flatten(jets)

        m3 = jets[:,1:4].sum()
        m4 = jets[:,0:4].sum()

        ones = ak.ones_like(jets.pt)

        imap = {
            "features": {
                "jetOrdinality":    ak.flatten(ak.local_index(jets, axis=1)),
                "jetPT": 		    flat_jets.pt - 2,
                "jetEta": 		    flat_jets.eta,
                "jetPhi": 		    flat_jets.phi,
                "jetBScore":    	flat_jets.btagDeepFlavB,
                "m3M": 			    ak.flatten(ones * m3.mass),
                "m3PT": 		    ak.flatten(ones * m3.pt),
                "m3Eta": 		    ak.flatten(ones * m3.eta),
                "m3Phi": 	        ak.flatten(ones * m3.phi),
                "m4M": 			    ak.flatten(ones * m4.mass),
                "m4PT":		        ak.flatten(ones * m4.pt),
                "m4Eta":		    ak.flatten(ones * m4.eta),
                "m4Phi":	        ak.flatten(ones * m4.phi)
            }
        }
        
        imap_concat = ak.concatenate([x[:, np.newaxis] for x in imap['features'].values()], axis=1)
        imap_scaled = (imap_concat - scaler.mean_) / scaler.scale_
        return (ak.values_astype(imap_scaled, "float32"),),{}

In [14]:
imap_concat - ak.Array(scaler.mean_)

<Array [[-2.05, 193, 0.156, ..., 0.503, 2.01], ...] type='20510 * 13 * float64'>

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(13,13)
        self.fc2 = nn.Linear(13,3)
    def forward(self,features=None):
        features = F.relu(self.fc1(features))
        features = F.softmax(self.fc2(features),dim=1)
        return features

In [16]:
model = jetAssignmentNN("traced_model.pt")

In [17]:
scores = model(events)

/usr/local/lib/python3.10/site-packages/coffea/ml_tools/helper.py:175: UserWarning: No format checks were performed on input!
  warnings.warn("No format checks were performed on input!")


In [15]:
ak.unflatten(scores[:,0], ak.num(jets))

<Array [[0.000372, 0.966, ..., 3.2e-14], ...] type='4212 * var * float32'>

In [16]:
ak.unflatten(scores[:,1], ak.num(jets))

<Array [[1, 0.0342, ..., 0.00696, 0.000148], ...] type='4212 * var * float32'>

In [21]:
highest_3_charg_score_idx = ak.argsort(ak.unflatten(scores[:,1], ak.num(jets)), axis=1)[:, -3:]
highest_stop_score_idx = ak.argsort(ak.unflatten(scores[:,0], ak.num(jets)), axis=1)[:, -1]

In [22]:
top_3_charg_score_sum = jets[highest_3_charg_score_idx].sum()
m3_top_3_nn_charg_score = top_3_charg_score_sum.mass

In [23]:
nn4 = top_3_charg_score_sum + jets[ak.singletons(highest_stop_score_idx)]

In [26]:
stop_jets = jets[ak.singletons(highest_stop_score_idx)]

In [10]:
m14 = jets[:, 0:4].sum().mass

In [11]:
m14

<Array [1.39e+03, 1.41e+03, ..., 1.26e+03, 2.81e+03] type='4212 * float32'>

In [35]:
ak.unflatten(scores[:,0], ak.num(jets))

<Array [[0.000372, 0.966, ..., 3.2e-14], ...] type='4212 * var * float32'>

In [32]:
highest_3_charg_score_idx

<Array [[3, 0, 2], [3, 0, 2], ..., [...], [1, 3, 0]] type='4212 * var * int64'>

In [31]:
highest_stop_score_idx

<Array [1, 1, 0, 1, 0, 0, 1, 0, ..., 0, 0, 0, 2, 1, 0, 2] type='4212 * int64'>

In [18]:
high_charg_score_mask = ak.unflatten(scores[:,1] > 0.95, ak.num(jets))
highest_3_charg_score_idx = ak.argsort(ak.unflatten(scores[:,1], ak.num(jets)), axis=1)[:, -3:]
highest_stop_score_idx = ak.argsort(ak.unflatten(scores[:,0], ak.num(jets)), axis=1)[:, -1]

top_3_charg_score_sum = jets[highest_3_charg_score_idx].sum()
m3_top_3_nn_charg_score = top_3_charg_score_sum.mass

In [19]:
m3_top_3_nn_charg_score

<Array [863, 834, 787, 507, ..., 937, 941, 680, 2.55e+03] type='4212 * float32'>

In [27]:

ak.flatten((top_3_charg_score_sum + stop_jets).mass)

<Array [1.39e+03, 1.41e+03, ..., 1.26e+03, 2.81e+03] type='4212 * float32'>

In [25]:
ak.flatten(nn4.mass)

<Array [1.39e+03, 1.41e+03, ..., 1.26e+03, 2.81e+03] type='4212 * float32'>

In [47]:
stop_jet = jets[ak.singletons(highest_stop_score_idx)]